# Referensi : https://www.youtube.com/watch?v=NBuED2PivbY


# Download chromeDriver first: https://chromedriver.chromium.org/home

In [ ]:
!pip install pillow
!pip install selenium
!pip install requests

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import io
from PIL import Image
import time
import os
import sys

In [ ]:
def get_images_from_google(wd, googleimg_url, delay, max_images):
    def scroll_down(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(delay)
    
    wd.get(googleimg_url)

    image_urls = set()
    skips = 0
    
    reached_page_end = False
    last_height = wd.execute_script("return document.body.scrollHeight")
    
    while len(image_urls) + skips < max_images and not reached_page_end: 
        # stop finding element, if it has reached (max_images) or (end of page)
        
        scroll_down(wd)
        new_height = wd.execute_script("return document.body.scrollHeight")
        if last_height == new_height:
            reached_page_end = True 
        else:
            last_height = new_height

        thumbnails = wd.find_elements(By.CLASS_NAME, "Q4LuWd")
        
        for img in thumbnails[len(image_urls) + skips:max_images]:
            try:
                img.click()
                time.sleep(delay)
            except:
                continue 

            images = wd.find_elements(By.CLASS_NAME, "iPVvYb")
            for image in images:
                if image.get_attribute('src') in image_urls:
                    max_images += 1
                    skips += 1
                    break

                if image.get_attribute('src') and 'http' in image.get_attribute('src'):
                    image_urls.add(image.get_attribute('src'))
                    sys.stdout.write("\rFound {0}".format(len(image_urls)))
                    sys.stdout.flush()

    return image_urls

In [ ]:
def download_image(url, download_path, file_name):
    try:
        image_content = requests.get(url).content
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file)
        file_path = download_path + file_name

        with open(file_path, "wb") as f:
            image.save(f, "JPEG")

    except Exception as e:
        print(f'\n{url} FAILED -', e) 

# Specify these variables

* "download_folder_name" : in which folder you want to save your downloaded images 
* "MAX_IMAGES" : set limit to maximum number of images that you want to download
* "googleimg_url" : URL of google search image 

In [ ]:
download_folder_name = 'papua'
MAX_IMAGES = 5
googleimg_url = "https://www.google.com/search?q=baju+adat+papua&rlz=1C1CHBD_idID1025ID1026&source=lnms&tbm=isch&sa=X&ved=2ahUKEwj-4_Lrm4b_AhWa-TgGHQSjAFkQ_AUoAXoECAwQAw&biw=1280&bih=601&dpr=1.5"

In [ ]:
current_dir = os.getcwd()

CHROME_PATH = os.path.join(current_dir,'chromedriver.exe')
DL_PATH = os.path.join(current_dir, 
                       f'{download_folder_name}/')

if not os.path.exists(DL_PATH):
    os.makedirs(DL_PATH) 
    
wd = webdriver.Chrome(CHROME_PATH)

urls = get_images_from_google(wd, googleimg_url, 1, MAX_IMAGES) 

for i, url in enumerate(urls):
    download_image(url, DL_PATH, str(i) + ".jpg")
    
wd.quit()

In [ ]:
# save lists of image URLs
with open(f'{download_folder_name}_urls.txt','w') as f:
    f.write(str(urls))

# load lists of image URLs
# import ast
# with open(f'{download_folder_name}_urls.txt','r') as f:
#     old_urls = ast.literal_eval(f.read())